In [2]:
# Notebook analysising the results of Exp-00
# classic models vs fully trained FGNs from scratch
# how do they compare over random noise

In [3]:
from __future__ import print_function

In [4]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt

In [5]:
import torch
import GPUtil
import time

import numpy as np
from scipy import stats

import os
import re
import pickle

import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [6]:
!gpustat

crescent  Thu Mar 19 15:19:02 2020
[0] GeForce GTX 1080 | 47'C,   0 % |  1857 /  8119 MB | felix(717M) near(487M) felix(643M)
[1] GeForce GTX 1080 | 37'C,   0 % |  4353 /  8119 MB | near(4343M)


In [7]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using device:", device)

# manualy set cuda device
torch.cuda.set_device(0)

CUDA Available:  True
Using device: cuda


In [8]:
# directory Exp-00
exp_dir = '/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/2020_01_07_at_23:28:46/'

In [9]:
# Part 0 - stats
# checking all kinds of FGN have been trained

In [10]:
# stats
classic_model_list = []
fgn_model_list = []
for path, dir,files in os.walk(exp_dir):
    for file in files:
        if 'trained_classic_model_full.pth' in file:
            classic_model_list.append((path,file))
        if 'trained_converted_fgn_model_full.pth' in file:
            fgn_model_list.append((path,file))

In [11]:
print('number of trained classic models:', len(classic_model_list))
print('number of trained FGN models:', len(fgn_model_list))

number of trained classic models: 185
number of trained FGN models: 10360


In [12]:
hidden_layer_sizes = [0, 16, 64, 256, 1024]
hidden_layer_numbers  = [0, 1, 2, 3]
dropout_probs = [0.0, 0.1, 0.4]

In [13]:
# classic network counts
# count all params
layer_number_counts = {key:0 for key in hidden_layer_numbers}
layer_sizes_counts = {key:0 for key in hidden_layer_sizes}
dp_counts = {key:0  for key in dropout_probs}

# for each path
for path,_ in classic_model_list:
    # get params
    s1 = re.split('/', path)[-1]
    s2 = re.split('_|\[|\]', s1)
    s3 = re.split(',', s2[2])
    if s3==['']:
        number = 0 
        size = 0
    else:
        number = len(re.split(',', s2[2]))
        size = int(re.split(',', s2[2])[0])
    drop_prob = float(s2[4][2:])
    
    
    # add to dicts
    layer_number_counts[number]+=1
    layer_sizes_counts[size]+=1
    dp_counts[drop_prob]+=1

In [14]:
for number in layer_number_counts.keys():
    print('{} classic networks with {} hidden layers'.format(layer_number_counts[number],number))
    
for size in layer_sizes_counts.keys():
    print('{} classic networks with size {} hidden layers'.format(layer_sizes_counts[size],size))

for dp in dp_counts.keys():
    print('{} classic networks with drop prob {}'.format(dp_counts[dp],dp))

5 classic networks with 0 hidden layers
60 classic networks with 1 hidden layers
60 classic networks with 2 hidden layers
60 classic networks with 3 hidden layers
5 classic networks with size 0 hidden layers
45 classic networks with size 16 hidden layers
45 classic networks with size 1024 hidden layers
45 classic networks with size 64 hidden layers
45 classic networks with size 256 hidden layers
65 classic networks with drop prob 0.0
60 classic networks with drop prob 0.4
60 classic networks with drop prob 0.1


In [15]:
# FGN specific params
covar_types = ['sphere', 'diag']
ordinals = [0.5, 1.0, 2.0, 5.0]
lmbda_sigmas = [8e-08, 8e-07, 4e-06, 8e-06, 1.6e-05, 8e-05, 0.0008]

In [19]:
def extract_params_from_path(path):
    # extracts classic and fgn params from a path, to rebuild from state dict
    # 3 cases: classic model, from scratch model, converted model
    
    results_dict = {}
    
    # get params
    # s1 = last directory
    s1 = re.split('/', path)[-1]
    
    s2 = re.split('_|\[|\]', s1)
    s3 = re.split(',', s2[2])
    if s3==['']:
        number = 0 
        size = 0
    else:
        number = len(re.split(',', s2[2]))
        size = int(re.split(',', s2[2])[0])
    
    results_dict['hidden_layers_sizes'] = [size for _ in range(number)]
    
    drop_prob = float(s2[4][2:])
    results_dict['drop_p'] = drop_prob
    
    # fgn specific
    try:
        covar_type = s2[5]
        if covar_type in ['sphere', 'diag']:
            results_dict['covar_type'] = covar_type
    except:
        pass
    try:
        ordinal = float(s2[6][3:])
        if ordinal in [0.5, 1.0, 2.0, 5.0]:
            results_dict['ordinal'] = ordinal
    except:
        pass
    try:
        ls = float(s2[7][2:])
        if ls in [8e-08, 8e-07, 4e-06, 8e-06, 1.6e-05, 8e-05, 0.0008]:
            results_dict['lmbda_sigma'] = ls
    except:
        pass

    
    return results_dict

In [21]:
fgn_model_list[0]

('/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/2020_01_07_at_23:28:46/classic_hl[1024]_dp0.4_ite4/converted FGNs/converted_fgn_hl[1024]_dp0.4_sphere_ord1.0_ls8e-06',
 'trained_converted_fgn_model_full.pth')

In [ ]:
path = fgn_model_list[0][0]
print(path)
s1 = re.split('/', path)[-1]
print(s1)
s2 = re.split('_|\[|\]', s1)
print(s2)

In [18]:
# fgn network counts
# count all params
layer_number_counts = {key:0 for key in hidden_layer_numbers}
layer_sizes_counts = {key:0 for key in hidden_layer_sizes}
dp_counts = {key:0  for key in dropout_probs}
covar_types_count = {key:0  for key in covar_types}
ordinals_counts = {key:0  for key in ordinals}
lmbda_sigmas_count = {key:0  for key in lmbda_sigmas}

# for each path
for path,_ in fgn_model_list:
    
    print(path)
    
    # get params
    s1 = re.split('/', path)[-1]
    s2 = re.split('_|\[|\]', s1)
    s3 = re.split(',', s2[2])
    if s3==['']:
        number = 0 
        size = 0
    else:
        number = len(re.split(',', s2[2]))
        size = int(re.split(',', s2[2])[0])
        
        
    drop_prob = float(s2[4][2:])
    covar_type = s2[5]
    ordinal = float(s2[6][3:])
    ls = float(s2[7][2:])
    
    # add to dicts
    layer_number_counts[number]+=1
    layer_sizes_counts[size]+=1
    dp_counts[drop_prob]+=1
    covar_types_count[covar_type]+=1
    ordinals_counts[ordinal]+=1
    lmbda_sigmas_count[ls]+=1

/home/felix/Research/Adversarial Research/FGN---Research/Experiments/Exp-00/2020_01_07_at_23:28:46/classic_hl[1024]_dp0.4_ite4/converted FGNs/converted_fgn_hl[1024]_dp0.4_sphere_ord1.0_ls8e-06


ValueError: invalid literal for int() with base 10: 'hl'

In [ ]:
for number in layer_number_counts.keys():
    print('{} fgn networks with {} hidden layers'.format(layer_number_counts[number],number))
    
for size in layer_sizes_counts.keys():
    print('{} fgn networks with size {} hidden layers'.format(layer_sizes_counts[size],size))

for dp in dp_counts.keys():
    print('{} fgn networks with drop prob {}'.format(dp_counts[dp],dp))
    
for covar in covar_types_count.keys():
    print('{} fgn networks with {} covariance matrix'.format(covar_types_count[covar],covar))
    
for ordinal in ordinals_counts.keys():
    print('{} fgn networks with {} ordinal '.format(ordinals_counts[ordinal],ordinal))

for ls in lmbda_sigmas_count.keys():
    print('{} fgn networks with lmbda_sigma of {}'.format(lmbda_sigmas_count[ls],ls))

In [ ]:
# Part 0 - comparison of performance
# how do FGNs trained from scratch perform in accuracy over MNIST compared to classic NN

In [ ]:
# load MNIST data
batch_sizefor_train = 10000
batch_size_for_val = 10000
(mnist_train_loader, mnist_val_loader, mnist_test_loader) = fgnh.mnist_dataloaders(batch_size=batch_sizefor_train,
                                                                                   batch_size_for_val=batch_size_for_val)

In [ ]:
classic_models_test_acc = []
classic_models_train_acc = []

for classic_model_path, filename  in classic_model_list:
    print('Working on {}'.format(classic_model_path))
    
    # load train history
    train_history = pickle.load(open(classic_model_path + '/train_histories.pckl', 'rb')) 
    # add to list for histogram
    classic_models_test_acc.append(train_history['test_acc_hist'])
    classic_models_train_acc.append(train_history['train_acc_hist'])
    

In [ ]:
stats.describe(classic_models_test_acc)

In [ ]:
fgn_models_test_acc = []
fgn_models_train_acc = []

for fgn_model_path, filename  in fgn_model_list:
    print('Working on {}'.format(fgn_model_path))
    
    # load train history
    train_history = pickle.load(open(fgn_model_path + '/train_histories.pckl', 'rb')) 
    # add to list for histogram
    fgn_models_test_acc.append(train_history['test_acc_hist'])
    fgn_models_train_acc.append(train_history['train_acc_hist'])
    

In [ ]:
stats.describe(fgn_models_test_acc)

In [ ]:
classic_models_test_acc[0]

In [ ]:
# plot means with error bars
x = [0,1,2,3,4]
y = stats.describe(classic_models_test_acc)[2]
plt.plot(x,y, label='classic test mean', c='C0')
y = stats.describe(classic_models_test_acc)[1][0]
plt.plot(x,y, label='classic test min', linestyle='--', c='C0')
# for y in classic_models_test_acc:
#     plt.plot(x,y, linestyle='', marker='.', alpha=0.1, c='C0')


y = stats.describe(fgn_models_test_acc)[2]
plt.plot(x,y, label='fgn test mean', c='C1')
y = stats.describe(fgn_models_test_acc)[1][0]
plt.plot(x,y, label='fgn test min', linestyle='--', c='C1')
# for y in fgn_models_test_acc:
#     plt.plot(x,y, linestyle='', marker='.', alpha=0.01, c='C1')
    
plt.title('MNIST accuracy over training epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# generate some random noise
num_samples = 5000
# (actually done later)
# white_noise_dataloader = fgnh.mnist_random_dataloader(num_samples=5000, batch_size=10000)
# shuffled_noise_dataloader = fgnh.mnist_random_shuffled_dataloader(num_samples=5000, batch_size=10000)

In [ ]:
# show examples
x = white_noise_dataloader.dataset.tensors[0][0]
plt.imshow(x, cmap=plt.cm.get_cmap('Greys'))
plt.title('Fully Random Image')
plt.colorbar()
plt.show()

x = shuffled_noise_dataloader.dataset.tensors[0][0]
plt.imshow(x, cmap=plt.cm.get_cmap('Greys'))
plt.title('Random from Real Image Shuffled Pixels')
plt.colorbar()
plt.show()

In [ ]:
def extract_params_from_path(path):
    # extracts classic and fgn params from a path, to rebuild from state dict
    
    results_dict = {}
    
    # get params
    s1 = re.split('/', path)[-1]
    s2 = re.split('_|\[|\]', s1)
    s3 = re.split(',', s2[2])
    if s3==['']:
        number = 0 
        size = 0
    else:
        number = len(re.split(',', s2[2]))
        size = int(re.split(',', s2[2])[0])
    
    results_dict['hidden_layers_sizes'] = [size for _ in range(number)]
    
    drop_prob = float(s2[4][2:])
    results_dict['drop_p'] = drop_prob
    
    # fgn specific
    try:
        covar_type = s2[5]
        if covar_type in ['sphere', 'diag']:
            results_dict['covar_type'] = covar_type
    except:
        pass
    try:
        ordinal = float(s2[6][3:])
        if ordinal in [0.5, 1.0, 2.0, 5.0]:
            results_dict['ordinal'] = ordinal
    except:
        pass
    try:
        ls = float(s2[7][2:])
        if ls in [8e-08, 8e-07, 4e-06, 8e-06, 1.6e-05, 8e-05, 0.0008]:
            results_dict['lmbda_sigma'] = ls
    except:
        pass

    
    return results_dict

In [ ]:
# quick test
extract_params_from_path(classic_model_list[0][0])

In [ ]:
def get_histogram_of_preds(model, dataloader, device, verbose=False):
    # given model
    # and a dataloader
    # returns a plt.hist of the predictions
    
    # send model to device and eval mode
    model.to(device)
    model.eval()
    
    preds_maxes = np.array([])
    with torch.no_grad():
        for x_data in dataloader:
            # in case the dataloader returns a x,y pair
#             print(len(x_data))
#             if len(x_data)==2:
            x_data = x_data[0]
            preds = model(x_data.to(device))
            # apply softmax for probs
            preds_softmax = np.array([np.exp(x)/sum(np.exp(x)) for x in preds.cpu().detach().numpy()])
            # only save max
            preds_maxes = np.concatenate((preds_maxes, np.max(preds_softmax, axis=1)))


    # quick description of the maxes
    stat = stats.describe(preds_maxes)
    if verbose: print(stat)
    # % of maxes above 0.5
    if verbose: print("percentage of confident (>0.5) predictions:",float(len([x for x in preds_maxes if x>=0.5])/float(len(preds_maxes))))

    # histogram
    hist = np.histogram(preds_maxes, bins=(np.arange(100)+1)/100.0)
    # to replot the original: plt.hist(hist[1][:-1], weights=hist[0], bins=hist[1])
    
    return(stat, hist)

In [ ]:
# Part 1 - Analysis

In [ ]:
# first the classic nets
classic_hist_list_white_noise = []
classic_hist_list_shuffled_noise = []

for classic_model_path, filename  in classic_model_list:
    print('Working on {}'.format(classic_model_path))
    
    # load model
    #try loading directly (will send to GPU it was trained on?)
    try:
        classic_model = torch.load(classic_model_path+'/'+filename)
        print('Loaded model directly')
    except Exception as e:
        print(e)
        print('loading model directly failed, rebuilding from state dict')
        params = extract_params_from_path(classic_model_path)
        print(params)
        # build model
        classic_model = fgnl.Feedforward_Classic_net(in_feats=28*28, 
                                                     out_feats = 10,
                                                     hidden_layer_sizes=params['hidden_layers_sizes'],
                                                     drop_p = params['drop_p']
                                                     )
        # load state_dict on CPU
        classic_model.load_state_dict(torch.load(classic_model_path+'/trained_classic_model_state_dict.pth'))
        
    # send to most free device
    # attempt to sent to GPU, else train over CPU
    model_sent_to_device = False
    sleep_time = 30
    while not model_sent_to_device and sleep_time<4800:
        # get free device
        device = torch.device('cuda')
        try:
            device_id = GPUtil.getFirstAvailable(order='memory', maxLoad=1.0, maxMemory=0.8, verbose=False)[0]
            # send to least used GPU
            print('Using GPU:', device_id)
            with torch.cuda.device(device_id):
                classic_model.to(device)
                model_sent_to_device=True

        except Exception as e:
            print(e)
            sleep_time = 1.66*sleep_time
            print('GPU error. Wait {}s and continue'.format(sleep_time))
            time.sleep(sleep_time)

    if not model_sent_to_device:
        print('Failed to send to GPU, using CPU')
        classic_model = torch.device('cpu')
        model.to(device)    
        
    # get new noise
    white_noise_dataloader = fgnh.mnist_random_dataloader(num_samples=num_samples, batch_size=10000)
    shuffled_noise_dataloader = fgnh.mnist_random_shuffled_dataloader(num_samples=num_samples, batch_size=10000)
    
    # get preds over white noise
    # ~4kb per 
    stat, hist = get_histogram_of_preds(classic_model, white_noise_dataloader, device)
    classic_hist_list_white_noise.append((classic_model_path, hist, stat))
    
    # get preds over shuffled noise
    stat, hist = get_histogram_of_preds(classic_model, shuffled_noise_dataloader, device)
    classic_hist_list_shuffled_noise.append((classic_model_path, hist, stat))
    

In [ ]:
def recursive_getsizeof(x, seen=set(), verbose=False):
    if id(x) in seen:
        return 0
    else:
        s = sys.getsizeof(x)
        if verbose: print(x,s)
        seen.add(id(x))
        # rec call
        try:
            for y in x:
                s += recursive_getsizeof(y, seen, verbose=verbose)
        except Exception as e:
            if verbose: print(e)
                
        return s

In [ ]:
recursive_getsizeof(classic_hist_list_white_noise[0], seen=set(), verbose=False)

In [ ]:
recursive_getsizeof(classic_hist_list_white_noise, seen=set(), verbose=False)

In [ ]:
# overall histogram - white noise
bins = classic_hist_list_white_noise[0][1][1] #will be the same for all
weights=[]
for path, hist, stat in classic_hist_list_white_noise:
    #weights
    w = hist[0]
    if len(weights) == 0:
        weights=w
    else:
        weights+=w
    
s = float(np.sum(weights))
weights = [w/s for w in weights]

plt.hist(bins[:-1], weights=weights, bins=bins)
plt.xticks((np.arange(10)+1)/10.0)
plt.grid(True)
plt.title("Classic-Net: Predictions over White Noise")
plt.show()
# % above 0.5
print("percentage of confident (>0.5) predictions:", np.sum(weights[49:]))

In [ ]:
# overall histogram - shuffled noise
bins = classic_hist_list_shuffled_noise[0][1][1] #will be the same for all
weights=[]
for path, hist, stat in classic_hist_list_shuffled_noise:
    #weights
    w = hist[0]
    if len(weights) == 0:
        weights=w
    else:
        weights+=w
    
s = float(np.sum(weights))
weights = [w/s for w in weights]

plt.hist(bins[:-1], weights=weights, bins=bins)
plt.xticks((np.arange(10)+1)/10.0)
plt.grid(True)
plt.title("Classic-Net: Predictions over Shuffled Noise")
plt.show()
# % above 0.5
print("percentage of confident (>0.5) predictions:", np.sum(weights[49:]))

In [ ]:
# histogram per param
# overall histogram - white noise
bins = classic_hist_list_shuffled_noise[0][1][1] #will be the same for all

for hl_size in hidden_layer_sizes:
    weights=[]
    for path, hist, stat in classic_hist_list_white_noise:
        if str(hl_size) in path:
            #weights
            w = hist[0]
            if len(weights) == 0:
                weights=w
            else:
                weights+=w

    s = float(np.sum(weights))
    weights = [w/s for w in weights]



    plt.hist(bins[:-1], weights=weights, bins=bins)
    plt.xticks((np.arange(10)+1)/10.0)
    plt.grid(True)
    plt.title("Classic-Net {} layer size: Predictions over White Noise".format(hl_size))
    plt.show()
    # % above 0.5
    print("percentage of confident (>0.5) predictions:", np.sum(weights[49:]))

In [ ]:
### 

In [ ]:
# set random noise num samples smaller because so many more FGNs
num_samples = 100

In [ ]:
# next the FGN
fgn_hist_list_white_noise = []
fgn_hist_list_shuffled_noise = []

for fgn_model_path, filename  in fgn_model_list:
    print('Working on {}'.format(fgn_model_path))
    
    # load model
    #try loading directly (will send to GPU it was trained on?)
    try:
        fgn_model = torch.load(fgn_model_path+'/'+filename)
        print('Loaded model directly')
    except Exception as e:
        print(e)
        print('loading model directly failed, rebuilding from state dict')
        params = extract_params_from_path(fgn_model_path)
        # build model
        fgn_model = fgnl.Feedforward_FGN_net(in_feats=28*28, 
                                             out_feats = 10,
                                             hidden_layer_sizes=params['hidden_layers_sizes'],
                                             drop_p = params['drop_p'],
                                             covar_type=params['covar_type'],
                                             non_lin=True)
        # load state_dict on CPU
        fgn_model.load_state_dict(torch.load(fgn_model_path+'/trained_fgn_model_state_dict.pth'))
        
    # send to most free device
    # attempt to sent to GPU, else train over CPU
    model_sent_to_device = False
    sleep_time = 30
    while not model_sent_to_device and sleep_time<4800:
        # get free device
        device = torch.device('cuda')
        try:
            device_id = GPUtil.getFirstAvailable(order='memory', maxLoad=1.0, maxMemory=0.8, verbose=False)[0]
            # send to least used GPU
            print('Using GPU:', device_id)
            with torch.cuda.device(device_id):
                fgn_model.to(device)
                model_sent_to_device=True

        except Exception as e:
            print(e)
            sleep_time = 1.66*sleep_time
            print('GPU error. Wait {}s and continue'.format(sleep_time))
            time.sleep(sleep_time)

    if not model_sent_to_device:
        print('Failed to send to GPU, using CPU')
        fgn_model = torch.device('cpu')
        model.to(device)    
        
    # get new noise
    white_noise_dataloader = fgnh.mnist_random_dataloader(num_samples=num_samples, batch_size=10000)
    shuffled_noise_dataloader = fgnh.mnist_random_shuffled_dataloader(num_samples=num_samples, batch_size=10000)
    
    # get preds over white noise
    # ~4kb per 
    stat, hist = get_histogram_of_preds(fgn_model, white_noise_dataloader, device)
    fgn_hist_list_white_noise.append((fgn_model_path, hist, stat))
    
    # get preds over shuffled noise
    stat, hist = get_histogram_of_preds(fgn_model, shuffled_noise_dataloader, device)
    fgn_hist_list_shuffled_noise.append((fgn_model_path, hist, stat))
    

In [ ]:
# overall histogram - white noise
bins = fgn_hist_list_white_noise[0][1][1] #will be the same for all
weights=[]
for path, hist, stat in fgn_hist_list_white_noise:
    #weights
    w = hist[0]
    if len(weights) == 0:
        weights=w
    else:
        weights+=w
    
s = float(np.sum(weights))
weights = [w/s for w in weights]

plt.hist(bins[:-1], weights=weights, bins=bins)
plt.xticks((np.arange(10)+1)/10.0)
plt.grid(True)
plt.title("FGNs: Predictions over White Noise")
plt.show()
# % above 0.5
print("percentage of confident (>0.5) predictions:", np.sum(weights[49:]))

In [ ]:
# overall histogram - shuffled noise
bins = fgn_hist_list_shuffled_noise[0][1][1] #will be the same for all
weights=[]
for path, hist, stat in fgn_hist_list_shuffled_noise:
    #weights
    w = hist[0]
    if len(weights) == 0:
        weights=w
    else:
        weights+=w
    
s = float(np.sum(weights))
weights = [w/s for w in weights]

plt.hist(bins[:-1], weights=weights, bins=bins)
plt.xticks((np.arange(10)+1)/10.0)
plt.grid(True)
plt.title("FGNs: Predictions over Shuffled Noise")
plt.show()
# % above 0.5
print("percentage of confident (>0.5) predictions:", np.sum(weights[49:]))

In [ ]:
# histogram per param
# overall histogram - white noise
bins = classic_hist_list_shuffled_noise[0][1][1] #will be the same for all

for ordinal in ordinals:
    weights=[]
    for path, hist, stat in fgn_hist_list_shuffled_noise:
        if str(ordinal) in path:
            #weights
            w = hist[0]
            if len(weights) == 0:
                weights=w
            else:
                weights+=w

    s = float(np.sum(weights))
    weights = [w/s for w in weights]



    plt.hist(bins[:-1], weights=weights, bins=bins)
    plt.xticks((np.arange(10)+1)/10.0)
    plt.grid(True)
    plt.title("FGN with ordinal {}: Predictions over Shuffled Noise".format(ordinal))
    plt.show()
    # % above 0.5
    print("percentage of confident (>0.5) predictions:", np.sum(weights[49:]))

In [ ]:
# histogram per param
# overall histogram - shuffled noise
bins = classic_hist_list_shuffled_noise[0][1][1] #will be the same for all

for ls in lmbda_sigmas:
    weights=[]
    for path, hist, stat in fgn_hist_list_shuffled_noise:
        if str(ls) in path:
            #weights
            w = hist[0]
            if len(weights) == 0:
                weights=w
            else:
                weights+=w

    s = float(np.sum(weights))
    weights = [w/s for w in weights]



    plt.hist(bins[:-1], weights=weights, bins=bins)
    plt.xticks((np.arange(10)+1)/10.0)
    plt.grid(True)
    plt.title("FGN with lmbda_sigma {}: Predictions over Shuffled Noise".format(ls))
    plt.show()
    # % above 0.5
    print("percentage of confident (>0.5) predictions:", np.sum(weights[49:]))

In [ ]:
# histogram per param
# overall histogram - shuffled noise
bins = classic_hist_list_shuffled_noise[0][1][1] #will be the same for all

for covar_type in covar_types:
    weights=[]
    for path, hist, stat in fgn_hist_list_shuffled_noise:
        if str(covar_type) in path:
            #weights
            w = hist[0]
            if len(weights) == 0:
                weights=w
            else:
                weights+=w

    s = float(np.sum(weights))
    weights = [w/s for w in weights]



    plt.hist(bins[:-1], weights=weights, bins=bins)
    plt.xticks((np.arange(10)+1)/10.0)
    plt.grid(True)
    plt.title("FGN with covar_type {}: Predictions over White Noise".format(covar_type))
    plt.show()
    # % above 0.5
    print("percentage of confident (>0.5) predictions:", np.sum(weights[49:]))